In [10]:
import requests
import xml.etree.ElementTree as ET

auth_key = "db8ed07d8514ae4697a742b9f0b3c0ff018270e9757ba44e021a968b603791c3"
url = f"http://data4library.kr/api/loanItemSrch?authKey={auth_key}&pageSize=5000"

# API 호출 및 XML 데이터 파싱
response = requests.get(url)
response.raise_for_status()
root = ET.fromstring(response.content)

# 'docs' 요소를 찾고, 그 하위의 'doc' 요소에서 책 제목(bookname) 추출
docs = root.find("docs")
if docs is not None:
    bookname_list = [doc.find("bookname").text.strip() for doc in docs.findall("doc") if doc.find("bookname") is not None]
else:
    bookname_list = []

print(bookname_list)


['나미야 잡화점의 기적 :히가시노 게이고 장편소설', '엄마를 부탁해 :신경숙 장편소설', '7년의 밤 :정유정 장편소설', '(趙廷來 大河小說) 太白山脈', '뚱딴지 명언 산책', '채식주의자:한강 연작소설', '미움받을 용기 :자유롭고 행복한 삶을 위한 아들러의 가르침', '아몬드손원평 장편소설', '틀려도 괜찮아', '82년생 김지영 :조남주 장편소설', '사피엔스 :유인원에서 사이보그까지, 인간 역사의 대담하고 위대한 질문', '강아지똥', '정글만리 :조정래 장편소설', '소년이 온다 :한강 장편소설', '책 먹는 여우', '마법의 설탕 두 조각', '연금술사', '박사가 사랑한 수식', '멈추면, 비로소 보이는 것들 :혜민 스님과 함께하는 내 마음 다시보기', '1Q84 :무라카미 하루키 장편소설', '종의 기원 =정유정 장편소설 /The origin of species', '(추리 천재) 엉덩이 탐정', '난장이가 쏘아올린 작은 공:조세희 소설집', '(추리 천재) 엉덩이 탐정', '살인자의 기억법 :김영하 장편소설', '(추리 천재) 엉덩이 탐정', '수박 수영장 :안녕달 그림책', '구해줘:기욤 뮈소 장편소설', '불편한 편의점 :김호연 장편소설', '언어의 온도 :말과 글에는 나름의 따뜻함과 차가움이 있다', '두근두근 내 인생 :김애란 장편소설', '정의란 무엇인가', '(추리 천재) 엉덩이 탐정', '설민석의 한국사 대모험', '(조선의 마지막 황녀) 덕혜옹주 :권비영 장편소설', '모모', '용의자 X의 헌신', '만희네 집', '초록 아이 그린링', '해리포터와 마법사의 돌', '내일은 실험왕 :본격 대결 과학실험 만화', '창문 넘어 도망친 100세 노인 :요나스 요나손 장편소설', '위저드 베이커리 =구병모 장편소설 /Wizard bakery', '시간을 파는 상점 :김선영 장편소설', '빅 픽처 :더글라스 케네디 장편소설', '아프니까 청춘이다 :인생 앞에 홀로 선 젊은 그대에게', '28 :정유정 장편소설', '돼지책', '설

In [11]:
pip install selenium beautifulsoup4 pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm

# Selenium Grid의 Hub URL
GRID_URL = 'http://localhost:4444/wd/hub'


# 책 소개글을 담을 리스트 초기화
book_introductions = []

# Selenium Grid를 사용한 WebDriver 설정
options = webdriver.ChromeOptions()
driver = webdriver.Remote(command_executor=GRID_URL, options=options)

# tqdm을 사용하여 진행 상황 표시
for bookname in tqdm(bookname_list, desc="Processing books", unit="book"):
    try:
        # 교보문고 사이트 열기
        driver.get('https://www.kyobobook.co.kr/')

        # 검색창에 책 제목 입력
        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "searchKeyword"))
        )
        search_box.clear()
        search_box.send_keys(bookname)

        # 검색 버튼 클릭
        search_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.btn_gnb_search'))
        )
        search_button.click()

        # 첫 번째 검색 결과 클릭
        first_result = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.prod_info'))
        )
        first_result.click()

        # 책 상세 페이지가 로드되기를 기다림
        time.sleep(3)  # 페이지 로드 시간을 충분히 줌

        # 페이지 소스 가져오기
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 책 소개글이 있는 div 찾기
        target_divs = soup.find_all('div', class_='info_text')

        if target_divs:
            extracted_text = ""

            # 모든 div의 텍스트를 하나로 결합 (br 태그는 제거)
            for div in target_divs:
                # br 태그 제거
                for br_tag in div.find_all('br'):
                    br_tag.decompose()
                # 텍스트 추출 및 결합
                extracted_text += div.get_text(strip=True)

            book_introductions.append(extracted_text)
        else:
            book_introductions.append('소개글을 찾을 수 없습니다.')

    except Exception as e:
        print(f"Error processing {bookname}: {e}")
        book_introductions.append('검색 결과가 없습니다.')

# 드라이버 종료
driver.quit()

# 데이터프레임 생성
df_results = pd.DataFrame({'책 제목': bookname_list, '책 소개': book_introductions})

# 데이터프레임을 엑셀 파일로 저장
OUTPUT_FILE = 'book_introductions.xlsx'
df_results.to_excel(OUTPUT_FILE, index=False, engine='openpyxl')

print(f"Results saved to {OUTPUT_FILE}")


Processing books:  37%|█████▏        | 1840/5000 [8:09:54<13:52:07, 15.80s/book]

Error processing 노인을 위한 나라는 없다: Alert Text: 나이 제한 상품입니다. 로그인후 이용하시기 바랍니다.
Message: unexpected alert open: {Alert text : 나이 제한 상품입니다. 로그인후 이용하시기 바랍니다.}
  (Session info: chrome=127.0.6533.120)
Stacktrace:
0   chromedriver                        0x000000010f2048b8 chromedriver + 5179576
1   chromedriver                        0x000000010f1fc2ea chromedriver + 5145322
2   chromedriver                        0x000000010ed732b0 chromedriver + 389808
3   chromedriver                        0x000000010edff31b chromedriver + 963355
4   chromedriver                        0x000000010ede1553 chromedriver + 841043
5   chromedriver                        0x000000010edb27f6 chromedriver + 649206
6   chromedriver                        0x000000010edb305e chromedriver + 651358
7   chromedriver                        0x000000010f1c7b20 chromedriver + 4930336
8   chromedriver                        0x000000010f1cca36 chromedriver + 4950582
9   chromedriver                        0x000000010f1cd105 chrom

Processing books: 100%|████████████████| 5000/5000 [22:18:00<00:00, 16.06s/book]


Results saved to book_introductions.xlsx
